In [1]:
from datasets import load_dataset
import matplotlib.pyplot as plt
import pandas as pd

dataset = load_dataset("zjunlp/FactCHD")

Generating train split:   0%|          | 0/51383 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6960 [00:00<?, ? examples/s]

In [2]:
df = pd.DataFrame(dataset["train"])
print(df.columns)

Index(['id', 'label', 'query', 'response', 'reason', 'type', 'domain',
       'subgraph_type', 'comparison_size', 'set_size', 'answer_size'],
      dtype='object')


In [20]:
def wrap_response(df):
    for index, row in df.iterrows():
        if row['label'] == 'FALSE':
            row['response'] = '<false>' + row['response'] + '</false>'

In [24]:
# Get 29,768 examples (where domain is common_wikipedia or common_wikidata)
filtered_df = df[df['domain'].isin(['common_wikipedia', 'common_wikidata'])]

# Get 500 examples of true query-response pairs
true = filtered_df[df['label'].isin(['FACTUAL'])]
true_trim = true[['label', 'query', 'response']]
true_trim['label'] = true_trim['label'].replace('FACTUAL', 'TRUE')
true_sample = true_trim.sample(n=500, replace=False, random_state=1) # same sample everytime (remove random_state to get different samples everytime) 

# Get 500 examples of false query-response pairs
false = filtered_df[df['label'].isin(['NON-FACTUAL'])]
false_trim = false[['label', 'query', 'response']]
false_trim['label'] = false_trim['label'].replace('NON-FACTUAL', 'FALSE')
false_sample = false_trim.sample(n=500, replace=False, random_state=1) # same sample everytime (remove random_state to get different samples everytime)

# Combine and shuffle (just cuz) to get 1000 training examples
combined_df = pd.concat([true_sample, false_sample], ignore_index=True)
shuffled_df = combined_df.sample(frac=1, random_state=1)

wrap_response(shuffled_df)

print(shuffled_df)

     label                                              query  \
507  FALSE  Among the given states and sovereign states, w...   
818  FALSE  Can you provide me with the box office earning...   
452   TRUE  Could you tell me which biopic Matt Damon star...   
368   TRUE  What is the total fertility rate of the United...   
242   TRUE  Could you tell me what position Mike Pence hel...   
..     ...                                                ...   
767  FALSE  Could you provide me with the birthdate of Ann...   
72    TRUE  Can you name a film based on literature that f...   
908  FALSE  Among the four places, which has the highest p...   
235   TRUE  Among the five sovereign states of China, Indi...   
37    TRUE  What was the population of the San Francisco B...   

                                              response  
507  <false>Among the given states and sovereign st...  
818  <false>Passengers grossed $303 million when it...  
452                    Matt Damon starred in a b

/tmp/ipykernel_143811/2811632539.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  true = filtered_df[df['label'].isin(['FACTUAL'])]
/tmp/ipykernel_143811/2811632539.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  true_trim['label'] = true_trim['label'].replace('FACTUAL', 'TRUE')
/tmp/ipykernel_143811/2811632539.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  false = filtered_df[df['label'].isin(['NON-FACTUAL'])]
/tmp/ipykernel_143811/2811632539.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs